# <div style='text-align: center; color: #2ecc71;'>BODYBYTE ELITE</div>
### <div style='text-align: center; color: #34495e;'>Large-Scale Personalized Workout & Diet Recommendation System</div>
---
Welcome to **BodyByte Elite**, now powered by a comprehensive database of **2,200+** exercise and nutrition entries. This system uses advanced filtering to deliver precise fitness and dietary recommendations.

In [16]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

display(HTML("""
<style>
    .card {
        box-shadow: 0 4px 8px 0 rgba(0,0,0,0.1);
        transition: 0.3s;
        width: 100%;
        border-radius: 12px;
        padding: 20px;
        margin-bottom: 20px;
        background-color: #ffffff;
        border-left: 5px solid #2ecc71;
    }
    .card:hover {
        box-shadow: 0 12px 24px 0 rgba(0,0,0,0.15);
        transform: translateY(-2px);
    }
    .header-box {
        background: linear-gradient(135deg, #273c75, #192a56);
        color: white;
        padding: 30px;
        border-radius: 20px;
        margin-bottom: 30px;
        text-align: center;
        box-shadow: 0 10px 20px rgba(0,0,0,0.1);
    }
    .metric-box {
        display: inline-block;
        width: 30%;
        margin: 1%;
        padding: 15px;
        background: #f8f9fa;
        border-radius: 12px;
        text-align: center;
        border: 1px solid #e9ecef;
    }
    .exercise-img {
        width: 100%;
        max-height: 200px;
        object-fit: cover;
        border-radius: 10px;
        margin-bottom: 15px;
    }
    .btn-custom {
        background: #44bd32 !important;
        color: white !important;
        font-weight: bold !important;
        border-radius: 25px !important;
    }
</style>
"""))

In [17]:
try:
    ex_df = pd.read_csv('exercises.csv')
    nu_df = pd.read_csv('nutrition.csv')
    print(f"✅ System Initialized with {len(ex_df)} exercises and {len(nu_df)} food items!")
except FileNotFoundError:
    print("❌ Dataset files missing!")

✅ System Initialized with 1100 exercises and 1100 food items!


In [18]:
def calculate_bmi(weight, height_cm):
    height_m = height_cm / 100
    bmi = weight / (height_m ** 2)
    if bmi < 18.5: category = "Underweight"
    elif 18.5 <= bmi < 25: category = "Normal"
    elif 25 <= bmi < 30: category = "Overweight"
    else: category = "Obese"
    return round(bmi, 2), category

def calculate_tdee(weight, height, age, gender, activity_level):
    if gender == 'Male':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    else:
        bmr = 10 * weight + 6.25 * height - 5 * age - 161
    mults = {'Sedentary': 1.2, 'Lightly Active': 1.375, 'Moderately Active': 1.55, 'Very Active': 1.725, 'Extra Active': 1.9}
    return round(bmr * mults.get(activity_level, 1.2))

In [19]:
def get_recommendations(user_profile):
    goal, level, tdee = user_profile['goal'], user_profile['level'], user_profile['tdee']
    
    # Exercise Filtering
    rec_ex = ex_df[(ex_df['Goal'] == goal) & (ex_df['Level'] == level)]
    if len(rec_ex) < 5: rec_ex = ex_df[ex_df['Goal'] == goal]
    
    # Diet Selection
    cal_target = tdee + 300 if goal == 'Muscle Gain' else (tdee - 500 if goal == 'Weight Loss' else tdee)
    rec_diet = nu_df[nu_df['Goal_Alignment'] == goal]
    if len(rec_diet) < 5: rec_diet = nu_df
    
    return rec_ex.sample(n=min(5, len(rec_ex))), rec_diet.sample(n=min(6, len(rec_diet))), cal_target

def generate_schedule(exercises):
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    sched = {}
    for day in days:
        if day in ['Wednesday', 'Sunday']:
            sched[day] = "Active Recovery (Yoga/Stretch)"
        else:
            sched[day] = ", ".join(exercises.sample(n=min(3, len(exercises)))['Exercise_Name'].tolist())
    return sched

In [20]:
output = widgets.Output()
style = {'description_width': 'initial'}

age = widgets.IntSlider(value=25, min=10, max=80, description='Age:', style=style)
gender = widgets.Dropdown(options=['Male', 'Female'], value='Male', description='Gender:', style=style)
height = widgets.IntText(value=175, description='Height (cm):', style=style)
weight = widgets.FloatText(value=70.0, description='Weight (kg):', style=style)
goal = widgets.Dropdown(options=['Muscle Gain', 'Weight Loss', 'Fitness', 'Body Building'], value='Fitness', description='Goal:', style=style)
activity = widgets.Dropdown(options=['Sedentary', 'Lightly Active', 'Moderately Active', 'Very Active', 'Extra Active'], value='Moderately Active', description='Activity Level:', style=style)
level = widgets.Dropdown(options=['Beginner', 'Intermediate', 'Advanced'], value='Beginner', description='Fitness Level:', style=style)
btn = widgets.Button(description='⚡ ANALYZE & RECOMMEND', layout=widgets.Layout(width='90%', height='50px'))
btn.add_class('btn-custom')

def run_app(b):
    with output:
        clear_output()
        bmi, cat = calculate_bmi(weight.value, height.value)
        tdee = calculate_tdee(weight.value, height.value, age.value, gender.value, activity.value)
        rec_ex, rec_diet, target = get_recommendations({'goal': goal.value, 'level': level.value, 'tdee': tdee})
        sched = generate_schedule(rec_ex)
        
        html = f"""
        <div class='header-box'>
            <h1 style='color:white;margin:0;'>BODYBYTE ELITE REPORT</h1>
            <p>Personalized Strategy for {goal.value}</p>
        </div>
        <div style='text-align: center; margin-bottom: 20px;'>
            <div class='metric-box'><h4>BMI</h4><b style='color:#2ecc71;font-size:24px;'>{bmi}</b><br>{cat}</div>
            <div class='metric-box'><h4>DAILY TDEE</h4><b style='color:#3498db;font-size:24px;'>{tdee}</b><br>kcal</div>
            <div class='metric-box'><h4>TARGET</h4><b style='color:#e67e22;font-size:24px;'>{target}</b><br>kcal/day</div>
        </div>
        <h3>🏋️ Top Exercise Picks (from 1,100+)</h3>
        <div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 15px;'>
        """
        for _, r in rec_ex.iterrows():
            html += f"<div class='card'><img src='{r['Image_URL']}' class='exercise-img'><h4>{r['Exercise_Name']}</h4><p><b>Focus:</b> {r['Target_Muscle']} | <b>Gear:</b> {r['Equipment']}</p></div>"
        
        html += "</div><h3>🥗 Nutrition Strategy</h3><div style='background:#f1f2f6;padding:20px;border-radius:15px;'><ul>"
        for _, r in rec_diet.iterrows():
            html += f"<li><b>{r['Food_Item']}:</b> {r['Calories']} kcal (P:{r['Protein']}g, C:{r['Carbs']}g)</li>"
        
        html += "</ul></div><h3>📅 Weekly Precision Plan</h3><table style='width:100%;background:white;border-radius:10px;overflow:hidden;'>"
        for d, w in sched.items():
            html += f"<tr style='border-bottom:1px solid #eee;'><td style='padding:12px;background:#f8f9fa;width:120px;'><b>{d}</b></td><td style='padding:12px;'>{w}</td></tr>"
        
        display(HTML(html + "</table>"))

btn.on_click(run_app)
display(widgets.VBox([widgets.HTML("<div style='background:#f8f9fa;padding:20px;border-radius:15px;border:1px solid #ddd;'>"), age, gender, height, weight, goal, activity, level, btn, widgets.HTML("</div>")]), output)

Output()